In [1]:
from utils.lyric_collect import get_whole_song_lyrics
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
import re
from urllib.parse import urljoin
import neologdn
import pickle
from datetime import datetime as dt


In [2]:
ROOT = Path("./").resolve().parent
SECRETS = ROOT / ".secrets"
DATA = ROOT / "data"
DATA.mkdir(exist_ok=True,parents=True)

In [3]:
res = requests.get("https://www.uta-net.com/ranking/artist/")
soup = BeautifulSoup(res.text, 'html.parser')

In [4]:
ranking = pd.DataFrame()
container = soup.select_one("#rank-total-tab")
for i,li in enumerate(container.select("ol.ranking-ol-table li")):
    rank = li.find("div", class_="text-center").get_text(strip=True)
    name = li.find_all("div")[1].get_text(strip=True)
    href = li.find_parent("a")["href"]  # アーティストページURL
    id = href.rstrip("/").split("/")[-1]
    ranking.loc[i,"rank"] = rank
    ranking.loc[i,"artist_id"] = id
    ranking.loc[i,"name"] = name

In [ ]:
LYRICS = DATA / "lyrics"
LYRICS.mkdir(exist_ok=True,parents=True)
execution_date = dt.today().strftime("%Y%m%d")
sample_n = 50
mode = 4

for row in ranking[["artist_id","name"]].iterrows():
    artist_id = row[1]["artist_id"]
    artist_name = row[1]["name"]
    if artist_id == "4002":
        continue
    else:
        data = get_whole_song_lyrics(artist_id, "artist", mode, sample_n,1)
        with open(LYRICS / f"{artist_name}_{execution_date}_mode{mode}_top{sample_n}.pkl", mode='wb') as f:
            pickle.dump(data, f)

back numberの歌詞一覧リスト125曲を取得します
全125曲完了                                                                                                             
嵐の歌詞一覧リスト361曲を取得します
全400曲完了                                                                                                             
RADWIMPSの歌詞一覧リスト186曲を取得します


In [ ]:
artist_ids = [
    "18526" # ミセス
    "684" # ミスチル
    "1475" # スピッツ
]

In [ ]:
get_whole_song_lyrics()